In [ ]:
import sys
from shutil import copy2
from imp import find_module
from glob import glob

import numpy as np
import pandas as pd
import statsmodels.api as sm

from bikewheelcalc import BicycleWheel, Rim, Hub, Spoke
import bikewheellib as bl
from doetools import LateralStiffnessDOE
from wheel_library import wheel_from_name

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from IPython.display import display

%matplotlib inline

pd.set_option('precision', 2)

## Load experimental data

In [ ]:
wf = pd.read_csv('../data/lateral_stiffness_tension/wheel_info.csv', index_col=0)

data_exp = pd.DataFrame()

for fname_t, fname_d in zip(glob('../data/lateral_stiffness_tension/[!~]*_tensions.csv'),
                            glob('../data/lateral_stiffness_tension/[!~]*_load-disp.csv')):
    
    # Load spoke tensions file
    t = pd.read_csv(fname_t, index_col=0)
    
    T_avg = t['Tension [N]'].mean()
    
    # Load load-displacement data
    ld = pd.read_csv(fname_d)
    
    # Calculate linear and quadratic stiffness
    lfit = sm.OLS(ld['Load [N]'].iloc[:-1],
                  sm.add_constant(ld['displacement [mm]'].iloc[:-1])).fit()
    
    K_lin = lfit.params['displacement [mm]']
    K_cint = lfit.conf_int(0.01).loc['displacement [mm]']
    K_err = (K_cint[1] - K_cint[0])/2
    
    # Error
    K_err = np.sqrt(K_err**2 + (K_lin*0.01)**2 +
                    (K_lin*0.001*25.4/ld['displacement [mm]'].max())**2)

    data_exp = data_exp.append(pd.Series({'Tension': T_avg,
                                          'K_lin': K_lin,
                                          'K_err': K_err}), ignore_index=True)

## Create ABAQUS simulations

In [ ]:
out_dir = '../data/doe/doe_Klat_tension'

# # Create ABAQUS simulations
# mrim = 0.538  # [kg]
# R = 0.304     # [m]
# GJ = 25.9     # [N-m^2]
# EI1 = 266.    # [N-m^2]
# EI2 = 113.    # [N-m^2]

doe = LateralStiffnessDOE(out_dir=out_dir,
                          opts={'spk_paired': False, 'spk_eltype': 'beam'})

# wh = BicycleWheel()
# wh.hub = Hub(diam1=0.058, width1=0.025+0.001)
# wh.rim = Rim(radius=R, area=mrim / (2*np.pi*2700.*R),
#              I11=GJ / 26.0e9, I22=EI2 / 69.0e9, I33=EI1 / 69.0e9, Iw=0.0 / 69.0e9,
#              young_mod=69.0e9, shear_mod=26.0e9)

# # Use the effective diameter for butted spokes
# wh.lace_radial(n_spokes=36, diameter=1.83e-3, young_mod=210e9, offset=0.00)

wh = wheel_from_name('std_elbows_in')

rr = bl.ModeMatrix(wh, N=36)

print bl.calc_buckling_tension(wh)
print rr.buckling_tension(smeared_spokes=True, coupling=False)

tension = np.linspace(0., 2000., 21)
for t in tension:
    jobname = 'Klat_T{:.0f}'.format(t)
    doe.add_experiment(wh, opts={'jobname': jobname, 'spk_T': t})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=2)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_lat_stiffness.py',
      dst=out_dir)

## Load ABAQUS results

In [ ]:
# Load DOE database
print 'Loading database...'
doe = LateralStiffnessDOE(out_dir=out_dir, db_file=out_dir+'/_doe_db.csv')

# Populate DOE database with results
print 'Extracting results...'
d = doe.extract_results()
print '\nDone'

In [ ]:
def rr_lateral_stiffness(r, smeared_spokes=True, buckling=True, coupling=True):
    w = doe.wheel_from_row(r)
    
    w.apply_tension(r['spk_T'] + 0.01)

    rr = bl.ModeMatrix(w, N=36)
    
    return rr.calc_lat_stiff(smeared_spokes=smeared_spokes,
                             buckling=buckling,
                             coupling=coupling)

doe.db['K_lat_cp_sm'] = doe.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=True), axis=1)
doe.db['K_lat_cp_ds'] = doe.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=False, coupling=True), axis=1)
doe.db['K_lat_nc_sm'] = doe.db.apply(lambda x: rr_lateral_stiffness(x, smeared_spokes=True, coupling=False), axis=1)

## Plot results

In [ ]:
def d_symm(y):
    n = len(y)
    return np.concatenate((y[n/2:], y[:(n/2+1)]))

def u3_theor(r, theta):
    'Get theroetical deformed shape'
    w = doe.wheel_from_row(r)
    
    w.apply_tension(r['spk_T'] + 0.01)

    rr = bl.ModeMatrix(w, N=36)
    
    # External forces
    F_ext = rr.calc_F_ext(np.array([0.]),
                          np.array([[1., 0., 0., 0.]]))

    K = rr.calc_K_rim(buckling=True) + rr.calc_K_spk(smeared_spokes=True)
    d = np.linalg.solve(K, F_ext)
    
    u3 = np.zeros(theta.shape)
    
    for i, t in enumerate(theta):
        u3[i] = rr.B_theta(t).dot(d)[0]
    
    return u3

# Damon Rinard's experiment
qturns = -np.arange(1, 11)
def1 = np.array([0.075, 0.071, 0.070, 0.070, 0.069,
                 0.068, 0.068, 0.069, 0.079, 0.130])

def2 = np.array([0.070, 0.069, 0.068, 0.067, 0.067,
                 0.067, 0.067, 0.068, 0.085, 0.157])

Klat_rinard_1 = 25.78*4.44822 / (def1*25.4)
Klat_rinard_2 = 25.78*4.44822 / (def2*25.4)
pf1 = np.polyfit(qturns[:-2:], Klat_rinard_1[:-2], 2)
pf2 = np.polyfit(qturns[:-2:], Klat_rinard_2[:-2], 2)

with plt.style.context(('seaborn-paper')):

    fig, ax = plt.subplots(ncols=3, figsize=(6.5, 2.25))
    
    # Theory, ABAQUS, and my experiments
    ax[0].plot(doe.db['spk_T']/1000, doe.db['K_lat_nc_sm'] / 1000, 'k', label='theory')
    ax[0].plot(doe.db['spk_T']/1000, doe.db['K_lat_abq'] / 1000, 'C0*', label='ABAQUS')
    ax[0].errorbar(x=data_exp['Tension']/1000, y=data_exp['K_lin'], fmt='C3o', label='experiments',
                   yerr=data_exp['K_err'])

    ax[0].set_xlabel('Spoke tension [kN]')
    ax[0].set_ylabel('Lat. stiffness [N/mm]')
    
    ax[0].set_xlim([0, 2])
    ax[0].set_xticks([0, 0.5, 1, 1.5, 2])
    
    ax[0].set_ylim([0., 125.])
    ax[0].set_yticks([0., 25, 50, 75, 100, 125])
    
    ax[0].legend(loc='best')
    
    
    cm = sns.color_palette('Reds', 4)
    # Damon Rinard's experiment
    ax[1].plot(qturns[:-2] / 4., np.polyval(pf1, qturns[:-2]), 'k--', label='trend')
    ax[1].errorbar(x=qturns / 4., y=Klat_rinard_1, label='Oct. 1999',
                   yerr=Klat_rinard_1*np.sqrt((0.002 / def1)**2 + 0.01**2),
                   fmt='o', color=cm[2])
    
    ax[1].plot(qturns[:-2] / 4., np.polyval(pf2, qturns[:-2]), 'k--', label='trend')
    ax[1].errorbar(x=qturns / 4., y=Klat_rinard_2, label='Nov. 1999',
                   yerr=Klat_rinard_2*np.sqrt((0.002 / def2)**2 + 0.01**2),
                   fmt='o', color=cm[1])
    
    ax[1].set_ylim([30., 70.])
    ax[1].set_xlim([-2.6, 0])
    
    ax[1].set_xlabel('Spoke nipple rotations')
    ax[1].set_ylabel('Lat. stiffness [N/mm]')
    
    ax[1].legend(loc='best')
    
    
    # Deformed shapes
    theta = np.linspace(-np.pi, np.pi, 37)
    
    ax[2].plot([], color='blue', marker='*', linestyle='none', label='ABAQUS')
    ax[2].plot([], color='black', label='theory')
    
    cm = mpl.cm.get_cmap('Blues')
    for t in [0., 500., 1000., 1500.]:
        r = doe.db[doe.db['spk_T'] == t].iloc[0]

        s = pd.read_csv(out_dir + '/' + r['jobname'] + '_shape.csv', header=1)
        u3 = s.loc[2].values[2:]
        
        ax[2].plot(theta, 100*u3_theor(r, theta) * 1000, color=cm(0.2 + 0.8*(t / 1500.)))
    
    ax[2].plot(theta, 100*d_symm(u3) * 1000., 'C0*')

    ax[2].set_xlabel(r'\$\theta\$')
    ax[2].set_ylabel(r'\$u/P\$ [mm/N] x 10')

    ax[2].set_xlim([-np.pi, np.pi])
    ax[2].set_xticks([-np.pi, -np.pi/2, 0., np.pi/2, np.pi])
    ax[2].set_xticklabels([r'\$-\pi\$', '','0', '', r'\$\pi\$'])
    
    ax[2].set_ylim([-2, 2.5])
    ax[2].set_yticks([-2, -1, 0., 1, 2])
    
    plt.tight_layout()
    plt.savefig('../figs/stress_analysis/_python_Klat_tension.pdf')

### Plot load-displacement curves

In [ ]:
cm = sns.color_palette('Reds', 7)

with plt.style.context(('seaborn-paper')):

    fig, ax = plt.subplots(figsize=(2.2, 1.8))
    

    for i, fname_d in enumerate(glob('../data/lateral_stiffness_tension/[!~]*_load-disp.csv')):

        ld = pd.read_csv(fname_d)

        u = ld['displacement [mm]'][:-1]  # Exclude the verification point
        P = ld['Load [N]'][:-1]

        lf = np.polyfit(u, P, 1)

        ax.plot(u, P, 'o', markersize=3, color=cm[i])
        ax.plot(u, np.polyval(lf, u), '-', linewidth=1, color=cm[i])
    
    ax.set_ylim([0., 120.])
    ax.set_xlim([0., 1.5])
    ax.set_xticks([0., 0.5, 1, 1.5])
    
    ax.set_xlabel('Lat. displacement [mm]')
    ax.set_ylabel('Load [N]')
    
    plt.tight_layout()
    plt.savefig('../figs/stress_analysis/_python_Klat_tension_ld.pdf')